# ELS Pricer - GPU Benchmark on Google Colab

**C++/CUDA 기반 2자산 Step-Down ELS Pricer 성능 테스트**

이 노트북은 다음을 테스트합니다:
- ✅ CPU 버전 (6가지 그리드 크기)
- ✅ GPU 버전 (6가지 그리드 크기)
- ✅ CPU vs GPU 성능 비교
- ✅ 버그 수정된 정확한 가격 계산

## 🔧 최근 업데이트 (2025-11-14)
- **조기상환 버그 수정**: 강제 상환으로 변경 (이전: 선택적)
- **타임스텝 버그 수정**: 만기 관측일 포함
- **가격 정확도 향상**: 111.7원 → 103.9원 (오차 7% → 0.5%)

## 📋 테스트 그리드 크기

| # | N1 | N2 | Nt | 총 Points | 설명 |
|---|----|----|----|-----------| |
| 1 | 100 | 100 | 200 | 2M | Small - Medium density |
| 2 | 100 | 100 | 1000 | 10M | Small - High time resolution |
| 3 | 200 | 200 | 200 | 8M | Large - Medium density |
| 4 | 200 | 200 | 1000 | 40M | Large - High time resolution |
| 5 | 400 | 400 | 200 | 32M | Very Large - Medium density |
| 6 | 400 | 400 | 1000 | 160M | Very Large - High time resolution |

---

## Step 1: GPU 환경 확인

In [ ]:
# GPU 확인
!nvidia-smi
print("\n" + "="*70)
!nvcc --version

## Step 2: 필요한 도구 설치

In [ ]:
# CMake와 빌드 도구 설치
!apt-get update -qq
!apt-get install -y -qq cmake build-essential

# 버전 확인
!cmake --version
!g++ --version | head -1

## Step 3: 프로젝트 업로드

**방법 1: 파일 직접 업로드 (추천)**
- `els-pricer-colab.tar.gz` 파일을 준비 (버그 수정 버전)
- 아래 셀 실행 후 파일 선택

**방법 2: Google Drive 마운트**
- Drive에 미리 업로드한 경우

**📌 중요**: 2025-11-14 이후 생성된 `els-pricer-colab.tar.gz` 파일을 사용하세요 (버그 수정 포함)

In [ ]:
# 방법 1: 파일 업로드
from google.colab import files
import os

print("📁 els-pricer-colab.tar.gz 파일을 업로드해주세요...")
print("   (버그 수정 버전: 2025-11-14 이후 생성)")
uploaded = files.upload()

# 압축 해제
if 'els-pricer-colab.tar.gz' in uploaded:
    !tar -xzf els-pricer-colab.tar.gz
    print("\n✓ 프로젝트 압축 해제 완료!")
    !ls -la els-pricer-cpp/
elif 'els-pricer-cpp.tar.gz' in uploaded:
    # 구버전 파일명도 지원
    !tar -xzf els-pricer-cpp.tar.gz
    print("\n✓ 프로젝트 압축 해제 완료! (구버전 파일명)")
    !ls -la els-pricer-cpp/
else:
    print("❌ els-pricer-colab.tar.gz 파일을 찾을 수 없습니다.")

In [ ]:
# (선택사항) 방법 2: Google Drive에서 가져오기
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/els-pricer-colab.tar.gz .
# !tar -xzf els-pricer-colab.tar.gz

## Step 4: 프로젝트 빌드 (GPU 버전)

In [ ]:
%cd els-pricer-cpp

# 빌드 디렉토리 생성
!rm -rf build
!mkdir -p build
%cd build

# CMake 설정 (GPU 포함)
print("🔧 CMake 설정 중...\n")
!cmake ..

print("\n" + "="*70)
print("🔨 컴파일 중 (병렬 빌드)...\n")
!make -j4

print("\n" + "="*70)
print("✓ 빌드 완료!")
!ls -lh els_pricer test_els

## Step 5: CPU vs GPU 벤치마크 실행

빌드가 완료되었습니다! 이제 CPU와 GPU 성능을 직접 비교하는 벤치마크를 실행합니다.

**테스트할 그리드:**
- 100×100×200, 100×100×1000
- 200×200×200, 200×200×1000
- 400×400×200, 400×400×1000

각 그리드마다 CPU와 GPU 모두에서 실행하고 어느 쪽이 더 빠른지 확인합니다.

In [ ]:
%cd /content/els-pricer-cpp/build

print("🚀 CPU vs GPU 종합 벤치마크 시작!\n")
print("="*70 + "\n")

# CPU vs GPU 비교 프로그램 실행 (6개 그리드 모두 테스트)
!./benchmark_cpu_vs_gpu

## Step 6: 결과 데이터 확인 및 분석

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CSV 파일 읽기
df = pd.read_csv('/content/els-pricer-cpp/build/cpu_vs_gpu_results.csv')

print("📊 CPU vs GPU 벤치마크 결과:\n")
print(df.to_string(index=False))

print("\n" + "═"*70)
print("📈 통계 요약:")
print(f"   평균 CPU 시간: {df['CPU_Time'].mean():.3f}초")
print(f"   평균 GPU 시간: {df['GPU_Time'].mean():.3f}초")
print(f"   평균 가속 비율: {df['Speedup'].mean():.2f}×")
print(f"   최대 가속 비율: {df['Speedup'].max():.2f}×")
print(f"   GPU 승리 횟수: {(df['Speedup'] > 1.0).sum()}/6")
print(f"   CPU 승리 횟수: {(df['Speedup'] < 1.0).sum()}/6")

# 각 그리드별 승자 표시
print("\n🏆 그리드별 승자:")
for idx, row in df.iterrows():
    grid_name = f"{row['Grid_N1']}×{row['Grid_N2']}×{row['Grid_Nt']}"
    if row['Speedup'] > 1.0:
        winner = f"GPU ✓ ({row['Speedup']:.2f}× 빠름)"
    elif row['Speedup'] < 1.0:
        winner = f"CPU ✓ ({1.0/row['Speedup']:.2f}× 빠름)"
    else:
        winner = "무승부"
    print(f"   {grid_name:16s} → {winner}")

## Step 7: 성능 비교 시각화

In [ ]:
# 그리드 라벨 생성
grid_labels = [f"{row['Grid_N1']}×{row['Grid_N2']}×{row['Grid_Nt']}" 
               for _, row in df.iterrows()]

# 플롯 생성
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('ELS Pricer: CPU vs GPU Performance Comparison', fontsize=16, fontweight='bold')

# 1. CPU vs GPU 실행 시간 비교
ax1 = axes[0, 0]
x = np.arange(len(grid_labels))
width = 0.35
bars1 = ax1.bar(x - width/2, df['CPU_Time'], width, label='CPU', color='steelblue', alpha=0.8)
bars2 = ax1.bar(x + width/2, df['GPU_Time'], width, label='GPU', color='coral', alpha=0.8)
ax1.set_xlabel('Grid Size', fontweight='bold')
ax1.set_ylabel('Time (seconds)', fontweight='bold')
ax1.set_title('Execution Time Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(grid_labels, rotation=45, ha='right')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 2. 가속 비율
ax2 = axes[0, 1]
colors = ['green' if s > 1.0 else 'red' for s in df['Speedup']]
bars = ax2.bar(grid_labels, df['Speedup'], color=colors, alpha=0.7)
ax2.axhline(y=1.0, color='black', linestyle='--', linewidth=1, label='Break-even')
ax2.set_xlabel('Grid Size', fontweight='bold')
ax2.set_ylabel('Speedup (×)', fontweight='bold')
ax2.set_title('GPU Speedup over CPU')
ax2.set_xticklabels(grid_labels, rotation=45, ha='right')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 가속비 값 표시
for i, (label, speedup) in enumerate(zip(grid_labels, df['Speedup'])):
    ax2.text(i, speedup + 0.5, f'{speedup:.1f}×', 
             ha='center', va='bottom', fontsize=9, fontweight='bold')

# 3. 가격 수렴성
ax3 = axes[1, 0]
ax3.plot(grid_labels, df['CPU_Price'], 'o-', label='CPU Price', markersize=8, linewidth=2)
ax3.plot(grid_labels, df['GPU_Price'], 's-', label='GPU Price', markersize=8, linewidth=2)
ax3.set_xlabel('Grid Size', fontweight='bold')
ax3.set_ylabel('ELS Price', fontweight='bold')
ax3.set_title('Price Convergence')
ax3.set_xticklabels(grid_labels, rotation=45, ha='right')
ax3.legend()
ax3.grid(alpha=0.3)

# 4. Points/second 처리 속도
ax4 = axes[1, 1]
cpu_throughput = df['Total_Points'] / df['CPU_Time'] / 1e6
gpu_throughput = df['Total_Points'] / df['GPU_Time'] / 1e6
x = np.arange(len(grid_labels))
bars1 = ax4.bar(x - width/2, cpu_throughput, width, label='CPU', color='steelblue', alpha=0.8)
bars2 = ax4.bar(x + width/2, gpu_throughput, width, label='GPU', color='coral', alpha=0.8)
ax4.set_xlabel('Grid Size', fontweight='bold')
ax4.set_ylabel('Throughput (M points/sec)', fontweight='bold')
ax4.set_title('Processing Throughput')
ax4.set_xticks(x)
ax4.set_xticklabels(grid_labels, rotation=45, ha='right')
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('cpu_vs_gpu_benchmark.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 그래프가 cpu_vs_gpu_benchmark.png로 저장되었습니다!")

## Step 8: 결과 파일 다운로드

In [ ]:
from google.colab import files

# CSV 결과 파일 다운로드
files.download('/content/els-pricer-cpp/build/cpu_vs_gpu_results.csv')

# 그래프 이미지 다운로드
files.download('cpu_vs_gpu_benchmark.png')

print("✅ 결과 파일 다운로드 완료!")
print("\n다운로드된 파일:")
print("  - cpu_vs_gpu_results.csv (벤치마크 결과 데이터)")
print("  - cpu_vs_gpu_benchmark.png (성능 비교 그래프)")

## Step 9: 가격 검증 테스트

실제 시장 파라미터로 ELS 가격을 계산하여 검증합니다.

**테스트 조건:**
- 기초자산 1: 변동성 15.2%, 배당 1.5%
- 기초자산 2: 변동성 40.4%, 배당 2.0%
- 상관계수: 0.61
- 무위험 이자율: 3.477%
- 만기: 3년 (6개월 간격 조기상환)
- 조기상환 베리어: 85%, 85%, 80%, 80%, 75%, 70%
- 낙인 베리어: 45%

**⚠️ 버그 수정 완료 (2025-11-14):**
- ✅ 조기상환이 강제(mandatory)로 수정됨 (이전: optional)
- ✅ 타임스텝 루프가 만기 관측일을 포함하도록 수정됨
- **예상 가격**: ~103.9원 (KI 미추적), ~93.9원 (KI 추적시)

In [ ]:
%cd /content/els-pricer-cpp/build

print("🔍 Running Price Validation Test...\n")
print("="*70 + "\n")

!./validate_price

print("\n" + "="*70)

## Step 10: Time Grid 스케일링 분석 (100×100 고정)

100×100 그리드 크기를 고정하고, Time Grid (Nt)를 변화시키면서 CPU와 GPU 성능이 어떻게 스케일링되는지 분석합니다.

**테스트 구성**: Nt = 100, 500, 1000, 2000, 3000, 4000, 5000

In [ ]:
%cd /content/els-pricer-cpp/build

print("🔬 Running Time Grid Scaling Benchmark...\n")
print("="*70 + "\n")

!./benchmark_nt_scaling

print("\n" + "="*70)

## Step 11: Nt Scaling 결과 시각화

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CSV 읽기
df_nt = pd.read_csv('/content/els-pricer-cpp/build/nt_scaling_results.csv')

print("📊 Time Grid Scaling Results (100×100 fixed):\n")
print(df_nt.to_string(index=False))

# 플롯 생성
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Time Grid (Nt) Scaling Analysis - 100×100 Fixed', fontsize=16, fontweight='bold')

# 1. CPU vs GPU 시간 비교 (선형)
ax1 = axes[0, 0]
ax1.plot(df_nt['Nt'], df_nt['CPU_Time'], 'o-', label='CPU', linewidth=2, markersize=8, color='steelblue')
ax1.plot(df_nt['Nt'], df_nt['GPU_Time'], 's-', label='GPU', linewidth=2, markersize=8, color='coral')
ax1.set_xlabel('Time Grid (Nt)', fontweight='bold', fontsize=12)
ax1.set_ylabel('Execution Time (seconds)', fontweight='bold', fontsize=12)
ax1.set_title('Execution Time vs Nt', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# 2. Speedup vs Nt
ax2 = axes[0, 1]
colors = ['green' if s > 1.0 else 'red' for s in df_nt['Speedup']]
ax2.bar(df_nt['Nt'], df_nt['Speedup'], color=colors, alpha=0.7, width=200)
ax2.axhline(y=1.0, color='black', linestyle='--', linewidth=1, label='Break-even (CPU = GPU)')
ax2.set_xlabel('Time Grid (Nt)', fontweight='bold', fontsize=12)
ax2.set_ylabel('Speedup (CPU / GPU)', fontweight='bold', fontsize=12)
ax2.set_title('GPU Speedup vs Nt', fontsize=14)
ax2.legend(fontsize=11)
ax2.grid(axis='y', alpha=0.3)

# Speedup 값 표시
for i, (nt, speedup) in enumerate(zip(df_nt['Nt'], df_nt['Speedup'])):
    ax2.text(nt, speedup + 0.05, f'{speedup:.2f}×', 
             ha='center', va='bottom', fontsize=9, fontweight='bold')

# 3. 타임스텝당 시간 (효율성)
ax3 = axes[1, 0]
cpu_per_step = (df_nt['CPU_Time'] / df_nt['Nt']) * 1000  # ms
gpu_per_step = (df_nt['GPU_Time'] / df_nt['Nt']) * 1000  # ms

ax3.plot(df_nt['Nt'], cpu_per_step, 'o-', label='CPU', linewidth=2, markersize=8, color='steelblue')
ax3.plot(df_nt['Nt'], gpu_per_step, 's-', label='GPU', linewidth=2, markersize=8, color='coral')
ax3.set_xlabel('Time Grid (Nt)', fontweight='bold', fontsize=12)
ax3.set_ylabel('Time per Timestep (ms)', fontweight='bold', fontsize=12)
ax3.set_title('Efficiency: Time per Timestep', fontsize=14)
ax3.legend(fontsize=11)
ax3.grid(alpha=0.3)

# 4. Processing Throughput
ax4 = axes[1, 1]
cpu_throughput = df_nt['Total_Points'] / df_nt['CPU_Time'] / 1e6
gpu_throughput = df_nt['Total_Points'] / df_nt['GPU_Time'] / 1e6

ax4.plot(df_nt['Nt'], cpu_throughput, 'o-', label='CPU', linewidth=2, markersize=8, color='steelblue')
ax4.plot(df_nt['Nt'], gpu_throughput, 's-', label='GPU', linewidth=2, markersize=8, color='coral')
ax4.set_xlabel('Time Grid (Nt)', fontweight='bold', fontsize=12)
ax4.set_ylabel('Throughput (M points/sec)', fontweight='bold', fontsize=12)
ax4.set_title('Processing Throughput vs Nt', fontsize=14)
ax4.legend(fontsize=11)
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('nt_scaling_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 그래프가 nt_scaling_analysis.png로 저장되었습니다!")

# 통계 요약
print("\n" + "═"*70)
print("📈 Nt Scaling 분석 요약:\n")
print(f"CPU 시간 증가율: {df_nt['CPU_Time'].iloc[-1] / df_nt['CPU_Time'].iloc[0]:.2f}× (Nt {df_nt['Nt'].iloc[0]} → {df_nt['Nt'].iloc[-1]})")
print(f"GPU 시간 증가율: {df_nt['GPU_Time'].iloc[-1] / df_nt['GPU_Time'].iloc[0]:.2f}× (Nt {df_nt['Nt'].iloc[0]} → {df_nt['Nt'].iloc[-1]})")
print(f"\n평균 Speedup: {df_nt['Speedup'].mean():.2f}×")
print(f"최소 Speedup: {df_nt['Speedup'].min():.2f}× (Nt={df_nt.loc[df_nt['Speedup'].idxmin(), 'Nt']:.0f})")
print(f"최대 Speedup: {df_nt['Speedup'].max():.2f}× (Nt={df_nt.loc[df_nt['Speedup'].idxmax(), 'Nt']:.0f})")

# 선형성 분석
cpu_slope = (df_nt['CPU_Time'].iloc[-1] - df_nt['CPU_Time'].iloc[0]) / (df_nt['Nt'].iloc[-1] - df_nt['Nt'].iloc[0])
gpu_slope = (df_nt['GPU_Time'].iloc[-1] - df_nt['GPU_Time'].iloc[0]) / (df_nt['Nt'].iloc[-1] - df_nt['Nt'].iloc[0])

print(f"\nCPU 타임스텝당 평균 시간: {cpu_slope * 1000:.4f} ms")
print(f"GPU 타임스텝당 평균 시간: {gpu_slope * 1000:.4f} ms")
print(f"효율 비율: GPU는 CPU의 {cpu_slope/gpu_slope:.2f}배 빠름 (타임스텝당)")

print("═"*70)

## 📌 요약

### 테스트 완료!

이 노트북에서:
1. ✅ GPU 환경 확인 (nvidia-smi, nvcc)
2. ✅ C++/CUDA 프로젝트 빌드 (CMake)
3. ✅ CPU vs GPU 종합 벤치마크 (6 grids)
4. ✅ 성능 비교 및 각 그리드별 승자 확인
5. ✅ 결과 시각화 및 다운로드

### 테스트한 그리드
- 100×100×200 (2M points)
- 100×100×1000 (10M points)
- 200×200×200 (8M points)
- 200×200×1000 (40M points)
- 400×400×200 (32M points)
- 400×400×1000 (160M points)

### 성능 (예상)
- **C++ CPU**: Python 대비 ~115-200× 빠름
- **C++ GPU**: CPU 대비 ~10-15× 추가 가속
- **총 가속**: Python 대비 ~1500-3000×

### 각 그리드별 승자 확인
벤치마크 출력에서 각 그리드마다:
- ✓ 표시로 승자 확인 (GPU ✓ 또는 CPU ✓)
- Speedup 값으로 얼마나 빠른지 확인
- 성능 크로스오버 지점 분석 (GPU가 더 빠른 시작점)

### 결과 파일
- `cpu_vs_gpu_results.csv`: 전체 벤치마크 데이터
- `cpu_vs_gpu_benchmark.png`: 성능 비교 시각화

---

**프로젝트**: ELS Pricer C++/CUDA  
**날짜**: 2025-11-14